In [81]:
# https://github.com/awjuliani/DeepRL-Agents/blob/master/Contextual-Policy.ipynb

In [82]:
import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense, Activation
from keras.optimizers import Adam, SGD
from keras import metrics
from keras import backend as K
import numpy as np

## The Contextual Bandits

In [254]:
class contextualBandit:
    def __init__(self):
        self.state = 0
        #List out our bandits. Currently arms 4, 2, and 1 (respectively) are the most optimal.
        self.bandits = np.array([[0.2,0,0.0,-5],[0.1,-5,1,0.25],[-5,5,5,5]])
        self.num_bandits = self.bandits.shape[0]
        self.num_actions = self.bandits.shape[1]
        
    def getBandit(self):
        self.state = np.random.randint(0,len(self.bandits)) #Returns a random state for each episode.
        return self.state
        
    def pullArm(self,action):
        #Get a random number.
        bandit = self.bandits[self.state,action]
        result = np.random.randn(1)
        if result > bandit:
            #return a positive reward.
            return 1
        else:
            #return a negative reward.
            return -1

## The Policy-Based Agent

In [255]:
def getAgentModel(input_size, output_size):
    ii = Input(shape=(input_size,))
#     oo = Dense(8, activation='relu')(ii)
    oo = Dense(output_size, activation='softmax')(ii)
    model = Model(ii, oo)
    return model

In [256]:
agent = getAgentModel(3, 4)
agent.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        (None, 3)                 0         
_________________________________________________________________
dense_16 (Dense)             (None, 4)                 16        
Total params: 16
Trainable params: 16
Non-trainable params: 0
_________________________________________________________________


In [260]:
def loss_custom(y_true,y_pred):
    # return -(K.log(y_pred)*y_true)
    return (K.log(y_pred)*y_true) * (K.log(y_pred)*y_true)

In [261]:
# my_loss = 'mean_squared_error'
my_loss = loss_custom
# opt = keras.optimizers.Adam(lr=0.001)
opt = keras.optimizers.SGD(lr=0.001)
agent.compile(optimizer=opt, loss=my_loss)

## Training the Agent

In [262]:
cBandit = contextualBandit()

In [263]:
total_episodes = 3000 #Set total number of episodes to train agent on.
total_reward = np.zeros([cBandit.num_bandits,cBandit.num_actions]) #Set scoreboard for bandits to 0.
e = 0.1 #Set the chance of taking a random action.

In [264]:
def get_one_hot(idx, size, val=1, def_vals=0):
    oh = np.full(size, def_vals)
    oh[idx] = val
    return oh
# UNIT TEST
print(get_one_hot(5, 10))

[0 0 0 0 0 1 0 0 0 0]


In [265]:
i = 0
DBG = 0
while i < total_episodes:
    s = cBandit.getBandit() #Get a state from the environment.
    if DBG:
        print("\nstate(s) : {}".format(s))
    
    #Choose either a random action or one from our network.
    if np.random.rand(1) < e:
        action = np.random.randint(cBandit.num_actions)
        if DBG:
            print("action raw (random) : {}".format(action))
    else:
        action = agent.predict(np.array([get_one_hot(s, cBandit.num_bandits)]))
        if DBG:
            print("action raw (agent) : {}".format(action))
    action_val = np.argmax(action)
    if DBG:
        print ("action_val : {}".format(action_val))
    
    #Get our reward for taking an action given a bandit.
    reward = cBandit.pullArm(action_val)
    if DBG:
        print("reward : {}".format(reward))
    
    #Update the network.
    state_arr = get_one_hot(s, cBandit.num_bandits)
    reward_arr = get_one_hot(action_val, cBandit.num_actions, val=reward, def_vals=0)
    if DBG:
        print("X : state = {}, Y : reward : {}".format(state_arr, reward_arr))
    agent.fit(np.array([state_arr]), np.array([reward_arr]),
                epochs=1,
                batch_size=1
                # shuffle=True
                # validation_data=(x_test, x_test)
               )
    i += 1

Epoch 1/1
1/1 [==============================] - 1s 513ms/step - loss: 0.2382
Epoch 1/1
1/1 [==============================] - 0s 996us/step - loss: 0.3785
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2473
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.2381
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.8491
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.7018
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2473
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2382
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3786
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2471
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.3781
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.7010
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2384


Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.3660
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3655
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2369
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.3651
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2368
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2417
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.8557
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.2417
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2414
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3649
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.7069
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.2371
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.364

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2363
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3544
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.8561
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2363
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3542
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3537
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2371
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.7090
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2365
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3536
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2369
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2367
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2364
Epoch 1/1


Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2327
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.2322
Epoch 1/1
1/1 [==============================] - 0s 1000us/step - loss: 0.2320
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2317
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.3480
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2316
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2313
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2327
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.3478
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.3473
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2313
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.7131
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2313

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2287
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2309
Epoch 1/1
1/1 [==============================] - 0s 1ms/step - loss: 0.2285
Epoch 1/1
1/1 [==============================] - 0s 996us/step - loss: 0.2306
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2283
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2280
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2305
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2302
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2300
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.3385
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.3380
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.3376
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2280
Ep

...

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2342
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1855
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.1639
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1854
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.7641
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2342
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.7627
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.2341
Epoch 1/1
1/1 [==============================] - 0s 999us/step - loss: 0.5319
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1639
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1858
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1857
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2342
Epoc

Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.1827
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1825
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.5322
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1824
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.1822
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1820
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.1628
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.5315
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.1627
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1820
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1818
Epoch 1/1
1/1 [==============================] - 0s 999us/step - loss: 0.1626
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1624
Ep

1/1 [==============================] - 0s 2ms/step - loss: 0.1802
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1590
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1588
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1587
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1586
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1584
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1583
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2312
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.1802
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2311
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1582
Epoch 1/1
1/1 [==============================] - 0s 4ms/step - loss: 0.1581
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.7619
Epoch 1/1
1/1 [=======

Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.1793
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.9199
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2274
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1574
Epoch 1/1
1/1 [==============================] - 0s 999us/step - loss: 0.2272
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2270
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1794
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1792
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1573
Epoch 1/1
1/1 [==============================] - 0s 997us/step - loss: 0.2269
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.1791
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2267
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.2264
Epoc

Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2229
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1778
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2227
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1777
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.2226
Epoch 1/1
1/1 [==============================] - 0s 3ms/step - loss: 0.1776
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1774
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.1543
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1773
Epoch 1/1
1/1 [==============================] - 0s 2ms/step - loss: 0.1771
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.1770
Epoch 1/1
1/1 [==============================] - 0s 998us/step - loss: 0.1768
Epoch 1/1
1/1 [==============================] - 0s 996us/step - loss: 0.2227
Epoc

In [266]:
print("CHECK AGENT PREDICTION")
res0 = agent.predict(np.array([[1, 0, 0]]))
print("agent predict 0 bandit's best : {} => {}".format(res0, np.argmax(res0)))
res1 = agent.predict(np.array([[0, 1, 0]]))
print("agent predict 1 bandit's best : {} => {}".format(res1, np.argmax(res1)))
res2 = agent.predict(np.array([[0, 0, 1]]))
print("agent predict 2 bandit's best : {} => {}".format(res2, np.argmax(res2)))

CHECK AGENT PREDICTION
agent predict 0 bandit's best : [[0.14559433 0.16711098 0.45716867 0.230126  ]] => 2
agent predict 1 bandit's best : [[0.17365348 0.28862932 0.104068   0.43364915]] => 3
agent predict 2 bandit's best : [[0.22892106 0.39178607 0.14570983 0.23358311]] => 1


In [267]:
print("THE ANSWER IS")
print("0 bandit : {} => {} is best".format(cBandit.bandits[0], np.argmin(cBandit.bandits[0])))
print("1 bandit : {} => {} is best".format(cBandit.bandits[1], np.argmin(cBandit.bandits[1])))
print("2 bandit : {} => {} is best".format(cBandit.bandits[2], np.argmin(cBandit.bandits[2])))

THE ANSWER IS
0 bandit : [ 0.2  0.   0.  -5. ] => 3 is best
1 bandit : [ 0.1  -5.    1.    0.25] => 1 is best
2 bandit : [-5.  5.  5.  5.] => 0 is best
